# Selection of the samples used for the heterogeneity analysis

In [1]:
import re
import cobra
import itertools
import pandas as pd
from IPython.lib.pretty import pretty

### 1.Load Used studies

In [2]:
species = 'escherichia_coli'
studies_file = 'sample_table.csv'

#### 1.1 Generate the study and the uptakes dictionaries

In [7]:
studies = ['Omics', 'Cra/Crp', 'Crp ARs', 'ICA']
entry_column = 'sample_id'
condition_columns = ['Base Media', 'Carbon Source (g/L)', 'Supplement']
metadata_filepath = '/'.join(['data',species, studies_file])
#generate study dicts with SRX entries and calculate uptake with Growth Rate
metadata = pd.read_csv(metadata_filepath)

#Load the universal and the organism model
org_model_name = 'iML1515.xml'
uni_model_name = 'universal_model_cobrapy.json'
org_model_filepath = '/'.join(['models', org_model_name])
uni_model_filepath = '/'.join(['models', uni_model_name])
org_cobra_model = cobra.io.read_sbml_model(org_model_filepath)
uni_cobra_model = cobra.io.load_json_model(uni_model_filepath)
org_cobra_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%


In [8]:
#generate df with all valid samples from selected studies
interest_columns = [ entry_column,
                     'study',
                     'condition',
                     'Strain Description',
                     'Base Media', 
                     'Carbon Source (g/L)', 
                     'Nitrogen Source (g/L)',
                     'Electron Acceptor',
                     'Supplement',
                     'Growth Rate (1/hr)'
                    ]

uptakes_dict = {}

studies_metadata = metadata[interest_columns]
valid_studies_metadata = studies_metadata.loc[studies_metadata['study'].isin(studies) &
                                              studies_metadata['Strain Description'].str.contains('MG1655') ] #only use data from strain MG1655

# Correct some metabolites names
met_names_to_replace =  ['N-acetylglucosamine', 'tyrosine', 'phenylalanine', 'methionine', 'glutamate', 'leucine', 'threonine', 'thiamine']
replace_with =  ['N-Acetyl-D-glucosamine', 'L-tyrosine', 'L-phenylalanine', 'L-methionine', 'L-glutamate', 'L-leucine', 'L-threonine', 'thiamin']
valid_studies_metadata = valid_studies_metadata.replace(regex = met_names_to_replace, value=replace_with)

display(valid_studies_metadata)


,sample_id,study,condition,Strain Description,Base Media,Carbon Source (g/L),Nitrogen Source (g/L),Electron Acceptor,Supplement,Growth Rate (1/hr)
48,p1k_00050,Omics,wt_glu,Escherichia coli K-12 MG1655,M9,glucose(4),NH4Cl(1),O2,L-glutamate (10mM),NaN
49,p1k_00051,Omics,wt_glu,Escherichia coli K-12 MG1655,M9,glucose(4),NH4Cl(1),O2,L-glutamate (10mM),NaN
50,p1k_00052,Omics,wt_gly,Escherichia coli K-12 MG1655,M9,glucose(4),NH4Cl(1),O2,glycine (10mM),NaN
51,p1k_00053,Omics,wt_gly,Escherichia coli K-12 MG1655,M9,glucose(4),NH4Cl(1),O2,glycine (10mM),NaN
52,p1k_00054,Omics,wt_thr,Escherichia coli K-12 MG1655,M9,glucose(4),NH4Cl(1),O2,L-threonine (10mM),NaN
...,...,...,...,...,...,...,...,...,...,...
179,p1k_00184,ICA,ura_pyr,Escherichia coli K-12 MG1655,M9,pyruvate(3.3),NH4Cl(1),O2,uracil (1 mM),0.27
180,p1k_00185,ICA,wt_glc,Escherichia coli K-12 MG1655,M9,glucose(2),NH4Cl(1),O2,NaN,0.63
181,p1k_00186,ICA,wt_glc,Escherichia coli K-12 MG1655,M9,glucose(2),NH4Cl(1),O2,NaN,0.63
184,p1k_00189,ICA,ade_glc,Escherichia coli K-12 MG1655,M9,glucose(2),NH4Cl(1),O2,adenine (100mg/L),0.78


In [9]:
#get the metabolite dict from universal model
#The statement *my_list would replace the list with its elements at the index positions. Thus, it unpacks the items of the lists.
all_target_mets = [*valid_studies_metadata[condition_columns[1]].tolist(), *valid_studies_metadata[condition_columns[2]].tolist()]

target_metabolites = set([ re.sub(r"\s?\(.*\)", "", met.lower()) for met in all_target_mets if type(met)==str ])


exchanges = [{exchanges : met.name} 
             for met in [mm
                         for mm in uni_cobra_model.metabolites
                         if any([m in mm.name.lower() for m in target_metabolites])]
             for exchanges in [r.id for r in met.reactions if len(r.metabolites)==1 and r.id.startswith('EX_')]]

exchanges = { k : v
              for exchange in exchanges
              for k,v in exchange.items()
            }

#curate it by only taking the lowest Levenshtein distances
exchanges_curated = {}
for met in target_metabolites:
    candidate_dict = {}
    for k, v in exchanges.items():
        if met.lower() in v.lower():
            candidate_dict[len(v)-len(met)] = k

    best_candidate = candidate_dict[min(candidate_dict.keys())]
    exchanges_curated[met] = uni_cobra_model.reactions.get_by_id(best_candidate)
print(exchanges_curated)
#with this translate the study dicts

{'fructose': <Reaction EX_fru_e at 0x7f128d2a3f40>, 'acetate': <Reaction EX_ac_e at 0x7f128d2a1de0>, 'adenine': <Reaction EX_ade_e at 0x7f12905549a0>, 'gluconate': <Reaction EX_glcn_e at 0x7f128d2eca60>, 'sorbitol': <Reaction EX_sbt__D_e at 0x7f1290560ee0>, 'l-glutamate': <Reaction EX_glu__L_e at 0x7f128d2ecd60>, 'l-arginine': <Reaction EX_arg__L_e at 0x7f128d2a27a0>, 'glycine': <Reaction EX_gly_e at 0x7f128d2ece20>, 'l-phenylalanine': <Reaction EX_phe__L_e at 0x7f128d2ef460>, 'd-ribose': <Reaction EX_rib__D_e at 0x7f128d2efb20>, 'galactose': <Reaction EX_gal_e at 0x7f1290558a00>, 'l-methionine': <Reaction EX_met__L_e at 0x7f129055b160>, 'glucose': <Reaction EX_glc__D_e at 0x7f128d2ec9a0>, 'n-acetyl-d-glucosamine': <Reaction EX_acgam_e at 0x7f1290554760>, 'cytidine': <Reaction EX_cytd_e at 0x7f1290556da0>, 'pyruvate': <Reaction EX_pyr_e at 0x7f1290560ca0>, 'glycerol': <Reaction EX_glyc_e at 0x7f128d2ecfa0>, 'thiamin': <Reaction EX_thm_e at 0x7f1290561900>, 'uracil': <Reaction EX_ura_e 

In [10]:
#generate the different study dicts:
#as i did in the putida analysis i will normalize the uptake
# by the number of carbons of the molecule and according to  
# the glucose uptake
glucose_uptake = abs(org_cobra_model.reactions.get_by_id(exchanges_curated['glucose'].id).lower_bound)

#check if all exchanges are included in the model and if not add the reactions

metabolites_to_add = [ met
                       for met in [rxn.reactants[0] for rxn in exchanges_curated.values() ]
                       if met not in org_cobra_model.metabolites 
                     ]

if len(metabolites_to_add) > 0:
  print('Before continue you need to add the following metabolites : %s' % ', '.join([r.id for r in metabolites_to_add]))

else:

  org_cobra_model.add_metabolites(metabolites_to_add)

  reactions_to_add = [ rxn
                      for rxn in exchanges_curated.values()
                      if rxn not in org_cobra_model.reactions
                    ]

  print('Need to add the following reactions : %s' % ', '.join([r.id for r in reactions_to_add]))

  org_cobra_model.add_reactions(reactions_to_add)

  #generate the carbon dictionary to normalize the uptake of each metabolite

  carbon_dict = { met : org_cobra_model.reactions.get_by_id(rxn.id).reactants[0].elements['C']
                  for met, rxn in exchanges_curated.items() }

  for study in studies:
      study_dict = {}
      uptake_dict = {}
      study_df = valid_studies_metadata.loc[metadata['study']==study]
      study_df.fillna('', inplace=True)
      unique_features = [ study_df[feature].unique().tolist() for feature in condition_columns ]
      feature_combination = list(itertools.product(*unique_features))
      for (media, carbon_source, supplement) in feature_combination:
          condition_df = study_df.loc[ (study_df[condition_columns[0]]==media) &
                                      (study_df[condition_columns[1]]==carbon_source) &
                                      (study_df[condition_columns[2]]==supplement)
                                    ]
          if len(condition_df)>1:
              for index, row in condition_df.iterrows():
                  media_tag = '--'.join([ re.sub(r"\s?\(.*\)", "", row[col].lower())  for col in condition_columns  ])
                  media_tag = [''.join([exchanges_curated[met].id, ':', str(glucose_uptake/carbon_dict['glucose']*carbon_dict[met])])
                              if met in exchanges_curated.keys()
                              else met
                              for met in media_tag.split('--') ]
                  study_dict['-'.join(media_tag)] = condition_df[entry_column].tolist()
      
                              
      print('_'.join([study, 'study']))
      print(pretty(study_dict))
      print('-------------------------------------------------------------------')

Need to add the following reactions : 
Omics_study
{'m9-EX_glc__D_e:10.0-EX_glu__L_e:8.333333333333334': ['p1k_00050',
  'p1k_00051'],
 'm9-EX_glc__D_e:10.0-EX_gly_e:3.3333333333333335': ['p1k_00052', 'p1k_00053'],
 'm9-EX_glc__D_e:10.0-EX_thr__L_e:6.666666666666667': ['p1k_00054',
  'p1k_00055']}
-------------------------------------------------------------------
Cra/Crp_study
{'m9-EX_ac_e:3.3333333333333335-': ['p1k_00082',
  'p1k_00083',
  'p1k_00086',
  'p1k_00087'],
 'm9-EX_fru_e:10.0-': ['p1k_00084', 'p1k_00085', 'p1k_00088', 'p1k_00089'],
 'm9-EX_glc__D_e:10.0-': ['p1k_00090', 'p1k_00091']}
-------------------------------------------------------------------
Crp ARs_study
{'m9-EX_fru_e:10.0-': ['p1k_00105', 'p1k_00116', 'p1k_00117', 'p1k_00118'],
 'm9-EX_glyc_e:5.0-': ['p1k_00106',
  'p1k_00107',
  'p1k_00108',
  'p1k_00109',
  'p1k_00110',
  'p1k_00111',
  'p1k_00112',
  'p1k_00113',
  'p1k_00114',
  'p1k_00115',
  'p1k_00122',
  'p1k_00123',
  'p1k_00124'],
 'm9-EX_glc__D_e:10.

/tmp/ipykernel_101928/936769647.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  study_df.fillna('', inplace=True)


#### *P.putida KT2440* studies

In [1]:
carbon_study = {   'm9-EX_cit_e:7.44' : ['SRX4552613',
                                         'SRX4552614',
                                         'SRX4552615'],
                   'm9-EX_fer_e:2.91' : ['SRX4552616',      #Experimentally measured in 0.1 glu
                                         'SRX4552617',
                                         'SRX4552618'],
                   'm9-EX_glc_e:7.44' : ['SRX4552619',      #Experimentally measured in 0.1 glu
                                         'SRX4552621'],
                   'm9-EX_ser__L_e:14.88' : ['SRX4552622',
                                             'SRX4552623',
                                             'SRX4552624'] }  



aromatics_study = { 'm9-EX_T4hcinnm_e:4.04' : ['SBRG_UNeb__coum__1',            #Experimentally measured in 0.1 glu
                                               'SBRG_UNeb__coum__2',
                                               'SBRG_UNeb__coum__3'], 
                    'm9-EX_T4hcinnm_e:2.02-EX_fer_e:1.4505' : ['SBRG_UNeb__coumferul__1', #Derived
                                                               'SBRG_UNeb__coumferul__2',
                                                               'SBRG_UNeb__coumferul__3'],  
                    'm9-EX_fer_e:2.91' : ['SBRG_UNeb__ferulate__1',            #Experimentally measured in 0.1 glu
                                          'SBRG_UNeb__ferulate__2',
                                          'SBRG_UNeb__ferulate__3'],  
                    'm9-EX_glc_e:7.44' : ['SBRG_UNeb__glc__1',                 #Experimentally measured in 0.1 glu
                                          'SBRG_UNeb__glc__2',
                                          'SBRG_UNeb__glc__3'] }

muconate_dict = {  'm9-EX_glc_e:7.44' : ['KT2440_glu_1',                 #Experimentally measured in 0.1 glu
                                        'KT2440_glu_2',
                                        'KT2440_glu_3'],
                   'm9-EX_glcn_e:7.44':['KT2440_glc_1',                 
                                        'KT2440_glc_2',
                                        'KT2440_glc_3'],
                   'm9-EX_fru_e:1.32' :['KT2440_f_1',                   #Multiplying the ratio of glucose and fructose
                                        'KT2440_f_2',                   #fluxes found in 0.3 glu by the glu flux in 0.1
                                        'KT2440_f_3'],                  #glu
                   'm9-EX_glc_e:3.72-EX_glcn_e:3.72' : ['KT2440_gg_1',
                                                        'KT2440_gg_2',
                                                        'KT2440_gg_3'],
                   'm9-EX_glc_e:3.72-EX_fru_e:0.66':['KT2440_fg_1',
                                                        'KT2440_fg_2',
                                                        'KT2440_fg_3'],
                   'm9-EX_glc_e:2.48-EX_glcn_e:2.48-EX_fru_e:0.44' : ['KT2440_gfg_1',
                                                                      'KT2440_gfg_2',
                                                                      'KT2440_gfg_3'] }

study_dict = { 'm9-EX_glc_e:7.44' : ['SRX7195897',        #Experimentally measured in 0.1 glu
                                     'SRX7195898',
                                     'SRX7195899'],
               'm9-EX_T4hcinnm_e:4.04' : ['SRX7195900',
                                          'SRX7195901',
                                          'SRX7195902']}  #Experimentally measured in 0.1 glu


#### *E. coli K12* studies

**When there is no available data concerning uptakes of each metabolites** in each condition, we are going to assume an uptake rate which is going to be **normalized according to the total carbon uptake from glucose in minimal media for heterotrophic organisms**. So if glucose uptake is 10 $\frac{mMol}{gDW \centerdot h}$, then the total carbon uptake **will be $10 \centerdot 6 = 60 \frac{C\ mMol}{gDW \centerdot h}$**

Another consideration is the total carbon uptake of the model itself **when multiple carbon sources** are available. In this sense, we assume that the **total carbon uptake for each compound is equal to total carbon uptake from glucose in minimal media divided by the number of metabolites**. So, if there carbon sources are glucose and acetate (N=2), the uptake will be computed as follows:


$compound\ uptake = \Large\frac{total\ carbon\ uptake}{compound\ carbon\ number \centerdot number\ of\ compounds}$


-  **Glucose :**  $Glucose\ uptake =\frac{60}{6 \centerdot 2} = 5 \frac{mMol}{gDW \centerdot h}$
-  **Acetate :**  $Acetate\ uptake = \frac{60}{2 \centerdot 2} = 15 \frac{mMol}{gDW \centerdot h}$

In [ ]:
omics_study = {'m9-EX_glc__D_e:10.0-EX_glu__L_e:8.33': ['p1k_00050',
                                                        'p1k_00051'],
               'm9-EX_glc__D_e:10.0-EX_gly_e:3.33': ['p1k_00052',
                                                                   'p1k_00053'],
               'm9-EX_glc__D_e:10.0-EX_thr__L_e:6.66': ['p1k_00054',
                                                                     'p1k_00055']
              }


Cra_Crp_study =  { 'm9-EX_ac_e:3.33': ['p1k_00082',
                                       'p1k_00083'],
                   'm9-EX_ac_e:3.33-deletion_Cra': ['p1k_00086',  #Cra Deletion
                                                    'p1k_00087'],  #Cra Deletion
                   'm9-EX_fru_e:10.0': ['p1k_00084',
                                        'p1k_00085'], 
                   'm9-EX_fru_e:10.0-deletion_Cra': ['p1k_00088',  #Cra Deletion
                                                     'p1k_00089'],  #Cra Deletion
                   'm9-EX_glc__D_e:10.0-deletion_Cra': ['p1k_00090',  #Cra Deletion
                                                        'p1k_00091']  #Cra Deletion
                 }


Crp_ARs_study = {'m9-EX_fru_e:10.0': [ 'p1k_00105'],
                 'm9-EX_fru_e:10.0-deletion_crp': ['p1k_00116',   #crp deletion
                                                   'p1k_00117',   #crp deletion
                                                   'p1k_00118'],  #crp deletion
                 'm9-EX_glyc_e:5.0': [ 'p1k_00106',
                                       'p1k_00107'],
                 'm9-EX_glyc_e:5.0-deletion_Ar1': ['p1k_00108',  #Ar1 deletion
                                                   'p1k_00109'], #Ar1 deletion
                 'm9-EX_glyc_e:5.0-deletion_Ar1_Ar2': ['p1k_00110',  #Ar1 and Ar2 deletion
                                                       'p1k_00111',  #Ar1 and Ar2 deletion
                                                       'p1k_00112'], #Ar1 and Ar2 deletion
                 'm9-EX_glyc_e:5.0-deletion_Ar2': ['p1k_00113',  #Ar2 deletion
                                                   'p1k_00114',  #Ar2 deletion
                                                   'p1k_00115'], #Ar2 deletion
                 'm9-EX_glyc_e:5.0-deletion_crp': ['p1k_00122',  #crp deletion
                                                   'p1k_00123',  #crp deletion
                                                   'p1k_00124'], #crp deletion
                 'm9-EX_glc__D_e:10.0-deletion_crp': ['p1k_00119',   #crp deletion
                                                      'p1k_00120',   #crp deletion
                                                      'p1k_00121']   #crp deletion
                }




ICA_study = {'m9-EX_glc__D_e:10.0': [ 'p1k_00161',
                                      'p1k_00162',
                                      'p1k_00163',
                                      'p1k_00164',
                                      'p1k_00185',
                                      'p1k_00186'],
             'm9-EX_glc__D_e:10.0-EX_o2_e:0.0': ['p1k_00175',    #anaerobic condition (KNO3 as e- acceptor)
                                                 'p1k_00176'],    #anaerobic condition (KNO3 as e- acceptor)]
             'm9-EX_glc__D_e:10.0-EX_gthrd_e:16.67': ['p1k_00169',
                                                      'p1k_00170'],
             'm9-EX_glc__D_e:10.0-EX_met__L_e:8.33': ['p1k_00173',
                                                      'p1k_00174'],
             'm9-EX_glc__D_e:10.0-EX_ade_e:8.33': ['p1k_00189',
                                                   'p1k_00190'],
             'm9-EX_sbt__D_e:10.0-EX_arg__L_e:10.0': ['p1k_00165',
                                                      'p1k_00166'],
             'm9-EX_rib__D_e:8.33-EX_cytd_e:15.0': ['p1k_00167',
                                                    'p1k_00168'],
             'm9-EX_glcr_e:10.0-EX_leu__L_e:10.0': ['p1k_00171',
                                                    'p1k_00172'],
             'm9-EX_acgam_e:13.33-EX_phe__L_e:15.0': ['p1k_00177',
                                                      'p1k_00178'],
             'm9-EX_gal_e:10.0-EX_thm_e:20.0': ['p1k_00179',
                                                'p1k_00180'],
             'm9-EX_glcn_e:10.0-EX_tyr__L_e:15.0': ['p1k_00181',
                                                    'p1k_00182'],
             'm9-EX_pyr_e:5.0-EX_ura_e:6.67': ['p1k_00183',
                                               'p1k_00184']
            }